In [2]:
import os
import tarfile
import gzip
import tempfile
import pandas as pd
import numpy as np
import netCDF4 as nc
from pyhdf.SD import SD, SDC


def read_hdf(filename, var_name = "par"):
    with nc.Dataset(filename) as file:
        file.set_auto_mask(False)
        variables = {x: file[x][()] for x in file.variables}
    return variables[var_name]


df = dict()

month_name = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Loop through each tar file
for year in range(1997, 2024):
    # Open the tar file
    tar_filename = f'data/par.s.{year}.tar'
    df[year] = pd.DataFrame()
    with tarfile.open(tar_filename, "r:") as tar:
        # Loop through each member of the tar file
        month = 8 if year == 1997 else -1
        for member in tar.getmembers():
            month += 1
            # if month == 3 and year == 2022:
                # month += 1 # Missing April 2022 data
            # If it's a .hdf.gz file
            if member.name.endswith('.hdf.gz'):
                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file)

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
                        data = hdf_file.select('par')[:]
                        df[year][month_name[month]] = data.reshape(-1)

In [7]:
df[2000]

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2332795,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2332796,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2332797,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2332798,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [8]:
#Make sure that "cleaned.xlsx" is in the same folder as the notebook
cleaned = pd.read_excel("cleaned.xlsx", index_col=0).dropna(subset = ["Lat", "Long"]) #Drop rows with no coordinates
cleaned

,Location,Cruise,Stn/Loc,Lat,Long,Depth,Cr_%,Cr_2pie,Cr_QF,Cr_nmol/kg,...,Filtered_2SEM,Filtered_Cr,Cr(III)_nmol/kg,Cr(III)_SD,Cr(III)_QF,Cr(III) filt out,Seawater,Source,Notes,Year
1,ETNP,TT145,2,21,-132,0.0,NaN,NaN,NaN,NaN,...,NaN,3.684864,0.097561,NaN,1.0,NaN,No,"Murray et al., 1983",lat & long est from fig,1983.0
2,ETNP,TT145,2,21,-132,25.0,NaN,NaN,NaN,3.190219,...,NaN,NaN,0.000000,NaN,1.0,NaN,No,"Murray et al., 1983",lat & long est from fig,1983.0
3,ETNP,TT145,2,21,-132,50.0,NaN,NaN,NaN,3.132063,...,NaN,NaN,0.204878,NaN,1.0,NaN,No,"Murray et al., 1983",lat & long est from fig,1983.0
4,ETNP,TT145,2,21,-132,75.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,1.0,NaN,No,"Murray et al., 1983",lat & long est from fig,1983.0
5,ETNP,TT145,2,21,-132,100.0,NaN,NaN,NaN,3.424319,...,NaN,NaN,0.097561,NaN,1.0,NaN,No,"Murray et al., 1983",lat & long est from fig,1983.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,ETSP,GP16,13,-14,-99,1987.8,0.771818,0.040358,2.0,4.851725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nasemann unpublished,NaN,NaN
533,ETSP,GP16,13,-14,-99,3378.6,0.769107,0.035261,2.0,5.234698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nasemann unpublished,NaN,NaN
534,ETSP,GP16,13,-14,-99,3811.1,0.758318,0.037619,2.0,5.088152,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nasemann unpublished,NaN,NaN
538,East Pac Rise,NaN,"HYCY 10,11",12.833333,103.933333,2615.0,NaN,NaN,NaN,15.800000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeandel & Minster 1984,"Omit: 35 m from bottom, unfilt",1984.0


In [9]:
#lon       | lat
#-180: 0   | 90: 0
#0: 1080   | 0: 540
#180: 2160 | -90: 1080

import math

for month in month_name:
    for i in cleaned.index:
        year = cleaned.loc[i, "Year"]
        year = int(year) if year == year else "NaN"
        lat, lon = cleaned.loc[i, "Lat"], cleaned.loc[i, "Long"]
        if type(lat) == str:
            lat = float(lat.replace("−", "-"))
        if type(lon) == str:
            lon = float(lon.replace("−", "-"))
        lat = (90 - lat) * 6
        lon = (lon + 180) * 6
        idx = (math.floor(lat) - 1) * 2160 + math.floor(lon) - 1
        if (year not in df) or (month not in df[year].columns) or (df[year][month][idx] < 0):
            #cleaned.loc[i, "par_" + month] = df["Mean"][month][(math.floor(lat) - 1) * 2160 + math.floor(lon) - 1]
            cnt = 0
            for yy in range(1997, 2024):
                if (month in df[yy].columns) and (df[yy][month][idx] >= 0):
                    if cnt == 0:
                        cleaned.loc[i, "par_" + month] = df[yy][month][idx]
                    else:
                        cleaned.loc[i, "par_" + month] += df[yy][month][idx]
                    cnt += 1
            if cnt == 0:
                cleaned.loc[i, "par_" + month] = -9999 #Missing every year
            else:
                cleaned.loc[i, "par_" + month] /= cnt
        else:
            cleaned.loc[i, "par_" + month] = df[year][month][idx]
        
cleaned

,Location,Cruise,Stn/Loc,Lat,Long,Depth,Cr_%,Cr_2pie,Cr_QF,Cr_nmol/kg,...,par_Mar,par_Apr,par_May,par_Jun,par_Jul,par_Aug,par_Sep,par_Oct,par_Nov,par_Dec
1,ETNP,TT145,2,21,-132,0.0,NaN,NaN,NaN,NaN,...,42.469329,46.885249,50.027220,48.296849,47.044190,44.691645,45.060672,39.309451,31.748226,27.239342
2,ETNP,TT145,2,21,-132,25.0,NaN,NaN,NaN,3.190219,...,42.469329,46.885249,50.027220,48.296849,47.044190,44.691645,45.060672,39.309451,31.748226,27.239342
3,ETNP,TT145,2,21,-132,50.0,NaN,NaN,NaN,3.132063,...,42.469329,46.885249,50.027220,48.296849,47.044190,44.691645,45.060672,39.309451,31.748226,27.239342
4,ETNP,TT145,2,21,-132,75.0,NaN,NaN,NaN,NaN,...,42.469329,46.885249,50.027220,48.296849,47.044190,44.691645,45.060672,39.309451,31.748226,27.239342
5,ETNP,TT145,2,21,-132,100.0,NaN,NaN,NaN,3.424319,...,42.469329,46.885249,50.027220,48.296849,47.044190,44.691645,45.060672,39.309451,31.748226,27.239342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,ETSP,GP16,13,-14,-99,1987.8,0.771818,0.040358,2.0,4.851725,...,51.795353,46.124461,40.317097,36.774048,36.760710,40.878019,46.562569,50.204218,52.101290,53.537738
533,ETSP,GP16,13,-14,-99,3378.6,0.769107,0.035261,2.0,5.234698,...,51.795353,46.124461,40.317097,36.774048,36.760710,40.878019,46.562569,50.204218,52.101290,53.537738
534,ETSP,GP16,13,-14,-99,3811.1,0.758318,0.037619,2.0,5.088152,...,51.795353,46.124461,40.317097,36.774048,36.760710,40.878019,46.562569,50.204218,52.101290,53.537738
538,East Pac Rise,NaN,"HYCY 10,11",12.833333,103.933333,2615.0,NaN,NaN,NaN,15.800000,...,50.470869,50.746087,47.770271,43.712099,42.138975,43.441122,44.700527,44.837563,42.830360,41.271037


### Kind of no idea about the code, will check later.